### **Minimum Cost Flow Network**

This is utilized as a tool to achieve balanced matching in an observational study; wherein its core idea is to frame the matching process as a network flow problem. The following are as defined and idientified:

>**Nodes**: Represent individual patients in the Interstitial Cystitis Data Base (ICDB).
>
>**Edges**: Represent potential matches between a treated patient and a control patient. An edge exists only if the control patient was not yet treated at the time the treated patient received the treatment (this is the "risk set" aspect).
>
>**Cost**: The "cost" associated with each edge represents the dissimilarity or distance between the treated and control patients in terms of their pretreatment characteristics (symptoms in this case). The Mahalanobis distance is used as a measure of this dissimilarity.
>
>**Flow**: The "flow" represents whether or not a particular match is selected. A flow of 1 means the match is made; a flow of 0 means it is not.

***Steps***

1. **Setting up the Network**: The authors construct a network where nodes represent patients, and edges represent permissible matches (treated patient points to an untreated patient).

2. **Defining Edge Costs**: The cost of each edge is calculated based on the Mahalanobis distance between the pretreatment symptom profiles of the treated and control patients.

3. **Applying Minimum Cost Flow**: The minimum cost flow algorithm is then used to find the set of matches (edges with flow = 1) that minimizes the total dissimilarity (total cost) across all matched pairs. This ensures that the matched pairs are as similar as possible.

4. **Balancing**: A penalty function within the integer programming framework is used to encourage balance in the marginal distributions of symptoms between the treated and control groups. This means they're not just trying to find similar pairs, but also ensure that the overall symptom profiles of the treated and control groups are comparable.

